In [1]:
import pandas as pd

csv_files = [
    'D:/assignment 2/CSV1.csv',
    'D:/assignment 2/CSV2.csv',
    'D:/assignment 2/CSV3.csv',
    'D:/assignment 2/CSV4.csv'
]

all_texts = []

for file in csv_files:
    try:
        df = pd.read_csv(file)

        print(f"Column names in {file}: {df.columns}")

        text_column = None
        for col in df.columns:
            if 'text' in col.lower():
                text_column = col
                break

        if text_column is not None:
            texts = df[text_column].astype(str)
            all_texts.extend(texts)
        else:
            print(f"Warning: No text column found in {file}")

    except pd.errors.EmptyDataError:
        print(f"Warning: Empty file found - {file}")

if all_texts:
    combined_text = "\n".join(all_texts)
    output_file = 'D:/assignment 2/combined_txt.txt'
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(combined_text)

    print(f'Combined texts saved to {output_file}')
else:
    print('No valid text columns found in the CSV files.')


Column names in D:/assignment 2/CSV1.csv: Index(['Unnamed: 0', 'HADM_ID', 'SHORT-TEXT', 'ICD9_CODE', 'ICD9', 'Label'], dtype='object')
Column names in D:/assignment 2/CSV2.csv: Index(['Unnamed: 0', 'HADM_ID', 'TEXT', 'LABLE', 'entites', 'group'], dtype='object')
Column names in D:/assignment 2/CSV3.csv: Index(['HADM_ID', 'TEXT', 'ICD9_CODE', 'Label'], dtype='object')
Column names in D:/assignment 2/CSV4.csv: Index(['HADM_ID', 'TEXT', 'LABLE'], dtype='object')
Combined texts saved to D:/assignment 2/combined_txt.txt


In [2]:
from collections import Counter
import csv

file_path = r'D:\assignment 2\answer\Question 1\COMBINE_TEXT.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()

words = text.split()

word_counts = Counter(words)

top_30_words = word_counts.most_common(30)

for word, count in top_30_words:
    print(f'{word}: {count}')
    
csv_file_path = r'D:\assignment 2\answer\Question 1\top30_words.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Word', 'Count']) 
    csv_writer.writerows(top_30_words)
print(f'Top 30 words and their counts saved to {csv_file_path}')

and: 1855246
the: 1622439
to: 1551252
of: 1529701
was: 1463004
with: 1085118
a: 946459
on: 846446
for: 792433
in: 790563
is: 590654
mg: 589659
The: 497153
Tablet: 452050
no: 414216
or: 410997
PO: 408981
patient: 407361
at: 406540
(1): 381504
as: 375101
Sig:: 371957
**]: 371169
Name: 364713
-: 317701
He: 299487
.: 282627
One: 282478
BLOOD: 282462
were: 282251
Top 30 words and their counts saved to D:\assignment 2\answer\Question 1\top30_words.csv


In [3]:
from collections import Counter
from transformers import AutoTokenizer

def count_and_get_top_tokens(file_path, model_name, top_n=30):

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    token_counts = Counter(tokens)

    top_tokens = token_counts.most_common(top_n)

    top_words = [(token, count) for token, count in top_tokens]

    return top_words

file_path = r'D:\assignment 2\answer\Question 1\COMBINE_TEXT.txt'
model_name = 'bert-base-uncased'
top_words = count_and_get_top_tokens(file_path, model_name)

for token, count in top_words:
    print(f'{token}: {count}')


c:\Users\Acer\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [4]:
import spacy

nlp = spacy.load("en_ner_bc5cdr_md")
nlp.max_length = 133328033

def extract_entities(text):
    doc = nlp(text)
    entities = {"diseases": [], "drugs": []}
    
    for ent in doc.ents:
        if hasattr(ent,'label_'):
            if ent.label_ == "DISEASE":
                entities["diseases"].append(ent.text)
            elif ent.label_ == "DRUG":
                entities["drugs"].append(ent.text)
    
    return entities

file_path = r'D:\assignment 2\answer\Question 1\COMBINE_TEXT.txt'
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

chunk_size = 5000
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

result = {"diseases": [], "drugs": []}

for chunk in chunks:
    entities = extract_entities(chunk)
    result["diseases"].extend(entities["diseases"])
    result["drugs"].extend(entities["drugs"])
 
print("Diseases:", result["diseases"])
print("Drugs:", result["drugs"])


c:\Users\Acer\AppData\Local\Programs\Python\Python38\lib\site-packages\spacy\language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

model = BertForTokenClassification.from_pretrained("monologg/biobert_v1.1_pubmed", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")

labels = ["O", "B-Disease", "I-Disease"]

def extract_entities(text):
    
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        outputs = model(input_tensor)

    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    entities = []
    current_entity = None

    for token, label_id in zip(tokens, predictions):
        label = labels[label_id]
        if label.startswith("B-"):
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = {"label": label[2:], "text": [token]}
        elif label.startswith("I-"):
            if current_entity is not None:
                current_entity["text"].append(token)
        else:
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = None

    if current_entity is not None:
        entities.append(current_entity)

    extracted_entities = [{"label": entity["label"], "text": " ".join(entity["text"])} for entity in entities]
    
    return extracted_entities

file_path = r'D:\assignment 2\answer\Question 1\COMBINE_TEXT.txt'
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

result = extract_entities(text)

diseases = [entity["text"] for entity in result if entity["label"] == "Disease"]
drugs = [entity["text"] for entity in result if entity["label"] == "Drug"]

print("Diseases:", diseases)
print("Drugs:", drugs)
